In [0]:
#configurations
storageAccount = "week5assignment"   
accessKey = "<access-key>"
container= "week8"
mount_point = f"/mnt/{container}"
if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = f"wasbs://{container}@{storageAccount}.blob.core.windows.net",
        mount_point = mount_point,
        extra_configs = {f"fs.azure.account.key.{storageAccount}.blob.core.windows.net": accessKey}
    )
else:
    print(f"{mount_point} is already mounted.")

/mnt/week8 is already mounted.


In [0]:
display(dbutils.fs.ls("/mnt/week8"))

path,name,size,modificationTime
dbfs:/mnt/week8/yellow_tripdata_2018-01.parquet,yellow_tripdata_2018-01.parquet,123668517,1752389352000


In [0]:
df = spark.read.parquet("/mnt/week8/yellow_tripdata_2018-01.parquet")
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [0]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2018-01-01 00:21:05|  2018-01-01 00:24:23|              1|          0.5|         1|                 N|          41|          24|           2|        4.5|  0.5|    0.5|       0.

Query 1. Add a column named as "Revenue" into dataframe which is the sum of the below columns: 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'

In [0]:
from pyspark.sql.functions import col
df = df.withColumn("Revenue", 
    col("fare_amount") + col("extra") + col("mta_tax") +
    col("improvement_surcharge") + col("tip_amount") +
    col("tolls_amount") + col("total_amount")
)

Query 2. Increasing count of total passengers in New York City by area

In [0]:
from pyspark.sql.functions import sum as s
df.groupBy("PULocationID")\
  .agg(s("passenger_count").alias("totalPassengers"))\
  .orderBy("TotalPassengers", ascending=False).show()

+------------+---------------+
|PULocationID|totalPassengers|
+------------+---------------+
|         161|         573916|
|         237|         572988|
|         236|         550847|
|         230|         511603|
|         162|         490738|
|         186|         460129|
|         234|         455335|
|         170|         440565|
|          48|         438312|
|         142|         427097|
|         163|         391685|
|          79|         390878|
|         138|         381519|
|         239|         370578|
|         141|         340100|
|         132|         333826|
|         164|         332941|
|         107|         330597|
|          68|         324889|
|         100|         296869|
+------------+---------------+
only showing top 20 rows



Query 3. Realtime Average fare/total earning amount earned by 2 vendors 

In [0]:
df.groupBy("VendorID")\
  .agg(s("fare_amount").alias("totalFare"),
       s("total_amount").alias("totalEarnings"))\
  .show()

+--------+-------------------+--------------------+
|VendorID|          totalFare|       totalEarnings|
+--------+-------------------+--------------------+
|       1|4.599966776999971E7|5.8181947048458464E7|
|       2|6.126991590999834E7|  7.75306291266784E7|
+--------+-------------------+--------------------+



Query 4. Moving Count of payments made by each payment mode

In [0]:
df.groupBy("payment_type").count().orderBy("count", ascending=False).show()

+------------+-------+
|payment_type|  count|
+------------+-------+
|           1|6106416|
|           2|2599215|
|           3|  43204|
|           4|  11852|
+------------+-------+



Query 5. Highest two gaining vendor's on a particular date with no of passenger and total distance by cab

In [0]:
from pyspark.sql.functions import to_date
df = df.withColumn("trip_date", to_date("tpep_pickup_datetime"))
df_filtered = df.filter(col("trip_date") == "2018-01-15")
df_filtered.groupBy("VendorID").agg(
    s("total_amount").alias("totalEarnings"),
    s("passenger_count").alias("totalPassengers"),
    s("trip_distance").alias("totalDistance")
).orderBy(col("totalEarnings").desc()).show(2)

+--------+------------------+---------------+------------------+
|VendorID|     totalEarnings|totalPassengers|     totalDistance|
+--------+------------------+---------------+------------------+
|       2|2237888.1600015936|         266559| 439143.8800000009|
|       1|1568897.4000015007|         130018|300357.29999999865|
+--------+------------------+---------------+------------------+



Query 6. Most no of passenger between a route of two location.

In [0]:
df.groupBy("PULocationID", "DOLocationID").agg(
    s("passenger_count").alias("totalPassengers")
).orderBy(col("totalPassengers").desc()).show()


+------------+------------+---------------+
|PULocationID|DOLocationID|totalPassengers|
+------------+------------+---------------+
|         264|         264|         186705|
|         237|         236|          86737|
|         236|         236|          78691|
|         236|         237|          71752|
|         237|         237|          64499|
|         239|         238|          44479|
|         239|         142|          42383|
|         142|         239|          40365|
|         238|         239|          37758|
|         141|         236|          36201|
|         230|         186|          35109|
|         237|         162|          34384|
|         186|         230|          33942|
|         263|         236|          33661|
|         161|         237|          33641|
|         237|         161|          33593|
|         239|         239|          32184|
|          48|          48|          31495|
|          79|          79|          31263|
|         186|         161|     

Query 7. Get top pickup locations with most passengers in last 5/10 seconds.

In [0]:
from pyspark.sql.functions import unix_timestamp, max as m, lit
max_time = df.select(m("tpep_pickup_datetime")).first()[0]
df_with_unix = df.withColumn("pickup_unix", unix_timestamp("tpep_pickup_datetime"))
max_unix_time = spark.sql(f"SELECT unix_timestamp('{max_time}')").first()[0]
print("\n Top pickup locations in last 5 seconds:")
df_with_unix.filter(col("pickup_unix") >= (max_unix_time - 5)) \
    .groupBy("PULocationID") \
    .agg(s("passenger_count").alias("recentPassengers")) \
    .orderBy(col("recentPassengers").desc()) \
    .show()
print("\n Top pickup locations in last 10 seconds:")
df_with_unix.filter(col("pickup_unix") >= (max_unix_time - 10)) \
    .groupBy("PULocationID") \
    .agg(s("passenger_count").alias("recentPassengers")) \
    .orderBy(col("recentPassengers").desc()) \
    .show()


 Top pickup locations in last 5 seconds:
+------------+----------------+
|PULocationID|recentPassengers|
+------------+----------------+
|          48|               2|
+------------+----------------+


 Top pickup locations in last 10 seconds:
+------------+----------------+
|PULocationID|recentPassengers|
+------------+----------------+
|          48|               2|
+------------+----------------+

